<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Learning FourSquare API with Python</font></h1>

## Introduction

In this lab, you will learn in details how to make calls to the Foursquare API for different purposes. You will learn how to construct a URL to send a request to the API to search for a specific type of venues, to explore a particular venue, to explore a Foursquare user, to explore a geographical location, and to get trending venues around a location. Also, you will learn how to use the visualization library, Folium, to visualize the results.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
1. <a href="#item1">Foursquare API Search Function</a>    
2. <a href="#item2">Explore a Given Venue</a>   
3. <a href="#item3">Explore a User</a>  
4. <a href="#item4">Foursquare API Explore Function</a>  
5. <a href="#item5">Get Trending Venues</a>  
</font>
</div>

### Import necessary Libraries

In [1]:
# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# !conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Define Foursquare Credentials and Version

##### Make sure that you have created a Foursquare developer account and have your credentials handy

In [2]:
CLIENT_ID = '3CYZT4F5XGUGM1JY1XP0B4XK0XTA2BPMXTO3QROSRMJSSPIF' # your Foursquare ID
CLIENT_SECRET = 'C24OJ1Z0EEKAVUR005PSUFIFEGJKEKTSACUZRSPZDOI3LRWJ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3CYZT4F5XGUGM1JY1XP0B4XK0XTA2BPMXTO3QROSRMJSSPIF
CLIENT_SECRET:C24OJ1Z0EEKAVUR005PSUFIFEGJKEKTSACUZRSPZDOI3LRWJ


#### Let's again assume that you are staying at the Conrad hotel. So let's start by converting the Contrad Hotel's address to its latitude and longitude coordinates.

In [3]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

c:\users\willson\appdata\local\programs\python\python36\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


40.7149555 -74.0153365


<a id="item1"></a>

## 1. Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

#### Now, let's assume that it is lunch time, and you are craving Italian food. So, let's define a query to search for Italian food that is within 500 metres from the Conrad Hotel. 

In [4]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


#### Define the corresponding URL

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=3CYZT4F5XGUGM1JY1XP0B4XK0XTA2BPMXTO3QROSRMJSSPIF&client_secret=C24OJ1Z0EEKAVUR005PSUFIFEGJKEKTSACUZRSPZDOI3LRWJ&ll=40.7149555,-74.0153365&v=20180604&query=Italian&radius=500&limit=30'

#### Send the GET Request and examine the results

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5beace049fb6b711a3161447'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'primary': True,
      'shortName': 'Pizza'}],
    'delivery': {'id': '294544',
     'provider': {'icon': {'name': '/delivery_provider_seamless_20180129.png',
       'prefix': 'https://igx.4sqi.net/img/general/cap/',
       'sizes': [40, 50]},
      'name': 'seamless'},
     'url': 'https://www.seamless.com/menu/harrys-italian-pizza-bar-225-murray-st-new-york/294544?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=294544'},
    'hasPerk': False,
    'id': '4fa862b3e4b0ebff2f749f06',
    'location': {'address': '225 Murray St',
     'cc': 'US',
     'city': 'New York',
     'country': 'United States',
     'distance': 5

#### Get relevant part of JSON and transform it into a *pandas* dataframe

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",294544,/delivery_provider_seamless_20180129.png,https://igx.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/harrys-italian-p...,False,4fa862b3e4b0ebff2f749f06,225 Murray St,...,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,Harry's Italian Pizza Bar,v-1542114820
1,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f3232e219836c91c7bfde94,63 W Broadway,...,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71446, 'lng': ...",40.714460,-74.010086,10007,NY,Conca Cucina Italian Restaurant,v-1542114820
2,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",NaN,NaN,NaN,NaN,NaN,NaN,False,3fd66200f964a520f4e41ee3,124 Chambers St,...,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'label': 'display', 'lat': 40.71533713859952...",40.715337,-74.008848,10007,NY,Ecco,v-1542114820


In [8]:
dataframe['categories']

0    [{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...
1    [{'id': '4d4b7105d754a06374d81259', 'name': 'F...
2    [{'id': '4bf58dd8d48988d110941735', 'name': 'I...
Name: categories, dtype: object

#### Define information of interest and filter dataframe

In [9]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,US,New York,United States,NaN,58,"[225 Murray St, New York, NY 10282, United Sta...","[{'label': 'display', 'lat': 40.71521779064671...",40.715218,-74.014739,10282,NY,4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,US,New York,United States,NaN,446,"[63 W Broadway, New York, NY 10007, United Sta...","[{'label': 'display', 'lat': 40.71446, 'lng': ...",40.714460,-74.010086,10007,NY,4f3232e219836c91c7bfde94
2,Ecco,Italian Restaurant,124 Chambers St,US,New York,United States,btwn Church St & W Broadway,549,[124 Chambers St (btwn Church St & W Broadway)...,"[{'label': 'display', 'lat': 40.71533713859952...",40.715337,-74.008848,10007,NY,3fd66200f964a520f4e41ee3


#### Let's visualize the Italian restaurants that are nearby

In [10]:
dataframe_filtered.name

0          Harry's Italian Pizza Bar
1    Conca Cucina Italian Restaurant
2                               Ecco
Name: name, dtype: object

In [11]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<a id="item2"></a>

## 2. Explore a Given Venue
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

### A. Let's explore the closest Italian restaurant -- _Harry's Italian Pizza Bar_

In [12]:
venue_id = '4fa862b3e4b0ebff2f749f06' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=3CYZT4F5XGUGM1JY1XP0B4XK0XTA2BPMXTO3QROSRMJSSPIF&client_secret=C24OJ1Z0EEKAVUR005PSUFIFEGJKEKTSACUZRSPZDOI3LRWJ&v=20180604'

#### Send GET request for result

In [13]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'delivery', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'allowMenuUrlEdit': True,
 'attributes': {'groups': [{'count': 1,
    'items': [{'displayName': 'Price', 'displayValue': '$$', 'priceTier': 2}],
    'name': 'Price',
    'summary': '$$',
    'type': 'price'},
   {'count': 7,
    'items': [{'displayName': 'Credit Cards',
      'displayValue': 'Yes (incl. American Express)'}],
    'name': 'Credit Cards',
    'summary': 'Credit Cards',
    'type': 'payments'},
   {'count': 1,
    'items': [{'displayName': 'Outdoor Seating', 'displayValue': 'Yes'}],
    'name': 'Outdoor Seating',
    'summary': 'Outdoor Seating',
    'type': 'outdoorSeating'},
   {'count': 8,
    'items': [{'displayName': 'Brunch', 'displayValue': 'Brunch'},
     {'displayName': 'Lunch', 'displayValue': 'Lunch'},
     {'displayName': 'Dinner', 'displayValue': 'Dinner'},
     {'displayName': 'Happy Hour', 'displayValue': 'Happy Hour'}],
    'name': 'Menus',
    'summary': 'Happy Hour, Brunch & more',
    'type': 'serves'},
   {'count': 5,
    'items': [{'displayName': 'Bee

### B. Get the venue's overall rating

In [14]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.1


That is not a very good rating. Let's check the rating of the second closest Italian restaurant.

In [15]:
venue_id = '4f3232e219836c91c7bfde94' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


Since this restaurant has no ratings, let's check the third restaurant.

In [16]:
venue_id = '3fd66200f964a520f4e41ee3' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.4


Since this restaurant has a slightly better rating, let's explore it further.

### C. Get the number of tips

In [17]:
result['response']['venue']['tips']['count']

16

### D. Get the venue's tips
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`/tips?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**`&limit=`**LIMIT**

#### Create URL and send GET request. Make sure to set limit to get all tips

In [18]:
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5beace114434b90234414763'},
 'response': {'tips': {'count': 16,
   'items': [{'agreeCount': 2,
     'authorInteractionType': 'liked',
     'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
     'createdAt': 1521601350,
     'disagreeCount': 0,
     'id': '5ab1cb46c9a517174651d3fe',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
     'todo': {'count': 0},
     'type': 'user',
     'user': {'firstName': 'Nick',
      'gender': 'male',
      'id': '484542633',
      'lastName': 'El-Tawil',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg'}}}]}}}

#### Get tips and list of associated features

In [19]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

#### Format column width and display all tips

In [20]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites",2,0,5ab1cb46c9a517174651d3fe,Nick,El-Tawil,male,484542633


Now remember that because we are using a personal developer account, then we can access only 2 of the restaurant's tips, instead of all 15 tips.

<a id="item3"></a>

## 3. Search a Foursquare User
> `https://api.foursquare.com/v2/users/`**USER_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

### Define URL, send GET request and display features associated with user

In [21]:
user_id = '484542633' # user ID with most agree counts and complete profile

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION) # define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()

dict_keys(['id', 'firstName', 'lastName', 'gender', 'canonicalUrl', 'photo', 'friends', 'tips', 'homeCity', 'bio', 'contact', 'photos', 'type', 'mayorships', 'checkins', 'lists', 'lenses'])

In [22]:
print('First Name: ' + user_data['firstName'])
print('Last Name: ' + user_data['lastName'])
print('Home City: ' + user_data['homeCity'])

First Name: Nick
Last Name: El-Tawil
Home City: New York, NY


#### How many tips has this user submitted?

In [23]:
user_data['tips']

{'count': 246}

Wow! So it turns out that Nick is a very active Foursquare user, with more than 250 tips.

### Get User's tips

In [24]:
# define tips URL
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

,text,agreeCount,disagreeCount,id
0,"The best! I’m especially fond of the salmon burger, but I’ve had half of the menu and never been disappointed. There’s a reason this place is well known even outside of the Village!",1,0,5aec594b1f7440002c138612
1,I used to down a pint of chocolate like it was nothing back when I was bulking. Highly recommended!,1,0,5accc9f66fa81f196724807b
2,They serve coffee!!!!!!,1,0,5accc98c0313204c9d7ec157
3,"I’m a fan. In fact, I’m such a big fan, I want Taim to hire me to be their spokesman. Kind of like the Arabic Jared Fogle - but without the kid stuff.",1,0,5accbf033abcaf09a24612a0
4,The linguine with clams is on point 👌,1,0,5accbe3a911fc423730f3ed3
5,"Great for a quick, cheap lunch! Shorter lines than Chipotle too👌",1,0,5acbecb86fa81f1967e019b0
6,"Quick, cheap lunch that tastes good! Way shorter line than Chipotle, too.",1,0,5acbec70a0215b732e264fe8
7,"Great for a quick, cheap bite. I stop by when I’m in the neighborhood and don’t have much time to eat.",1,0,5acbec0c6fa81f1967dfeba3
8,You’re not a real New Yorker until you’ve shame-ordered Insomnia Cookies for delivery at 3am,1,0,5acbbd4eb1538e45373b07f5
9,Good for you yet still tasty! Clean green protein is my go-to after I hit the gym 💪,1,0,5acbbcda01235808d5d6dc75


#### Let's get the venue for the tip with the greatest number of agree counts

In [25]:
tip_id = '5ab5575d73fe2516ad8f363b' # tip id

# define URL
url = 'http://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}'.format(tip_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# send GET Request and examine results
result = requests.get(url).json()
print(result['response']['tip']['venue']['name'])
print(result['response']['tip']['venue']['location'])

Cowgirl
{'address': '519 Hudson St', 'crossStreet': 'at W 10th St', 'lat': 40.73375031678678, 'lng': -74.00628236760791, 'labeledLatLngs': [{'label': 'display', 'lat': 40.73375031678678, 'lng': -74.00628236760791}], 'postalCode': '10014', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['519 Hudson St (at W 10th St)', 'New York, NY 10014', 'United States']}


### Get User's friends

In [26]:
user_friends = json_normalize(user_data['friends']['groups'][0]['items'])
user_friends

""


Interesting. Despite being very active, it turns out that Nick does not have any friends on Foursquare. This might definitely change in the future.

### Retrieve the User's Profile Image

In [27]:
user_data

{'bio': 'https://www.tawil.team/nick-el-tawil/',
 'canonicalUrl': 'https://foursquare.com/nickeltawil',
 'checkins': {'count': 1, 'items': []},
 'contact': {},
 'firstName': 'Nick',
 'friends': {'count': 0,
  'groups': [{'count': 0,
    'items': [],
    'name': 'Other friends',
    'type': 'others'}]},
 'gender': 'male',
 'homeCity': 'New York, NY',
 'id': '484542633',
 'lastName': 'El-Tawil',
 'lenses': [],
 'lists': {'count': 2,
  'groups': [{'count': 0, 'items': [], 'type': 'created'},
   {'count': 0, 'items': [], 'type': 'followed'},
   {'count': 2,
    'items': [{'canonicalUrl': 'https://foursquare.com/nickeltawil/list/todos',
      'collaborative': False,
      'description': '',
      'editable': False,
      'id': '484542633/todos',
      'listItems': {'count': 0},
      'name': "Nick's Saved Places",
      'public': True,
      'type': 'todos',
      'url': '/nickeltawil/list/todos'},
     {'canonicalUrl': 'https://foursquare.com/nickeltawil/list/venuelikes',
      'collaborat

In [28]:
# 1. grab prefix of photo
# 2. grab suffix of photo
# 3. concatenate them using the image size  
Image(url='https://igx.4sqi.net/img/user/300x300/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg')

<a id="item4"></a>

## 4. Explore a location
> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**

#### So, you just finished your gourmet dish at Ecco, and are just curious about the popular spots around the restaurant. In order to explore the area, let's start by getting the latitude and longitude values of Ecco Restaurant.

In [29]:
latitude = 40.715337
longitude = -74.008848

#### Define URL

In [30]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=3CYZT4F5XGUGM1JY1XP0B4XK0XTA2BPMXTO3QROSRMJSSPIF&client_secret=C24OJ1Z0EEKAVUR005PSUFIFEGJKEKTSACUZRSPZDOI3LRWJ&ll=40.715337,-74.008848&v=20180604&radius=500&limit=30'

#### Send GET request and examine results

In [31]:
import requests

In [32]:
results = requests.get(url).json()
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around Ecco restaurant.'

#### Get relevant part of JSON

In [33]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'reasonName': 'globalInteractionReason',
    'summary': 'This spot is popular',
    'type': 'general'}]},
 'referralId': 'e-0-4af5d65ff964a52091fd21e3-0',
 'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/furniture_',
     'suffix': '.png'},
    'id': '4bf58dd8d48988d1f8941735',
    'name': 'Furniture / Home Store',
    'pluralName': 'Furniture / Home Stores',
    'primary': True,
    'shortName': 'Furniture / Home'}],
  'id': '4af5d65ff964a52091fd21e3',
  'location': {'address': '57 Warren St',
   'cc': 'US',
   'city': 'New York',
   'country': 'United States',
   'crossStreet': 'Church St',
   'distance': 73,
   'formattedAddress': ['57 Warren St (Church St)',
    'New York, NY 10007',
    'United States'],
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.71482437714839,
     'lng': -74.00940425461492}],
   'lat': 40.71482437714839,
   'lng': -74.00940425461492,
   'neighborhood': 'Tribeca',
   

#### Process JSON and convert it to a clean dataframe

In [34]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Korin,Furniture / Home Store,57 Warren St,US,New York,United States,Church St,73,"[57 Warren St (Church St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71482437714839, 'lng': -74.00940425461492}]",40.714824,-74.009404,Tribeca,10007,NY,4af5d65ff964a52091fd21e3
1,Takahachi Bakery,Bakery,25 Murray St,US,New York,United States,at Church St,179,"[25 Murray St (at Church St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71372322632139, 'lng': -74.00873266967818}]",40.713723,-74.008733,NaN,10007,NY,4c154c9a77cea593c401d260
2,Chambers Street Wines,Wine Shop,148 Chambers St,US,New York,United States,btwn West Broadway & Hudson St,88,"[148 Chambers St (btwn West Broadway & Hudson St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.715773063928374, 'lng': -74.00971823312332}]",40.715773,-74.009718,NaN,10007,NY,4adcf23cf964a520cc6221e3
3,Takahachi,Sushi Restaurant,145 Duane St,US,New York,United States,btwn W Broadway & Church St,146,"[145 Duane St (btwn W Broadway & Church St), New York, NY 10013, United States]","[{'label': 'display', 'lat': 40.71652647412374, 'lng': -74.00810108466207}]",40.716526,-74.008101,NaN,10013,NY,4a8f2f39f964a520471420e3
4,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,US,New York,United States,btwn Greenwich St & W Broadway,202,"[83 Murray St (btwn Greenwich St & W Broadway), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71478769908051, 'lng': -74.0111317502157}]",40.714788,-74.011132,NaN,10007,NY,54148bc6498ea7bb8c05b70a
5,Mulberry & Vine,Café,73 Warren St,US,New York,United States,btwn W Broadway & Greenwich St,117,"[73 Warren St (btwn W Broadway & Greenwich St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.71517693966315, 'lng': -74.01022747778285}]",40.715177,-74.010227,NaN,10007,NY,5171b5cc011cef9833bbb787
6,Nish Nūsh,Falafel Restaurant,88 Reade St,US,New York,United States,at Church St,97,"[88 Reade St (at Church St), New York, NY 10013, United States]","[{'label': 'display', 'lat': 40.71553710116416, 'lng': -74.00772452925565}]",40.715537,-74.007725,NaN,10013,NY,50ba9119e4b071a4bae6dc10
7,Equinox Tribeca,Gym,54 Murray Street,US,New York,United States,at W Broadway,159,"[54 Murray Street (at W Broadway), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.714057, 'lng': -74.009694}]",40.714057,-74.009694,NaN,10007,NY,4a6e331af964a52031d41fe3
8,Ten Over Ten,Nail Salon,112 Reade St,US,New York,United States,NaN,68,"[112 Reade St, New York, NY 10013, United States]","[{'label': 'display', 'lat': 40.71594125566931, 'lng': -74.00872053564494}]",40.715941,-74.008721,NaN,10013,NY,4ce2a27cd58c60fc0fa1a76f
9,Little Park,American Restaurant,85 W Broadway,US,New York,United States,at Chambers St,29,"[85 W Broadway (at Chambers St), New York, NY 10007, United States]","[{'label': 'display', 'lat': 40.715486585249735, 'lng': -74.00913313510836}]",40.715487,-74.009133,NaN,10007,NY,545c0436498e798e22ce4b2a


#### Let's visualize these items on the map around our location

In [35]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

<a id="item5"></a>

## 5. Explore Trending Venues
> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**

#### Now, instead of simply exploring the area around Ecco, you are interested in knowing the venues that are trending at the time you are done with your lunch, meaning the places with the highest foot traffic. So let's do that and get the trending venues around Ecco.

In [36]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5beace1e351e3d426e1e152e'},
 'response': {'venues': []}}

### Check if any venues are trending at this time

In [37]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [38]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 

### Visualize trending venues

In [39]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [40]:
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'

<a id="item6"></a>

### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson/). I hope you found this lab interesting and educational. Feel free to contact me if you have any questions!

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week2_LAB1).

<hr>
Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).